# nightscout/cgm-remote-monitor Github Dashboard <img src ="https://avatars2.githubusercontent.com/u/7661012?s=200&v=4" width="60px" alt="dotnet bot in space" align ="right">

In [1]:
display(HTML("<b style=\"color:blue\">Install NuGet packages</b>"));

Install NuGet packages

In [1]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"
using Octokit;
using NodaTime;
using NodaTime.Extensions;
using XPlot.Plotly;

Installing package Octokit, version 0.32.0..................done!

Successfully added reference to package Octokit, version 0.32.0

Installing package NodaTime, version 2.4.6....done!

Successfully added reference to package NodaTime, version 2.4.6

## Create a GitHub public API client

In [2]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

In [3]:
var today = SystemClock.Instance.InUtc().GetCurrentDate();
var startOfTheMonth = today.With(DateAdjusters.StartOfMonth);
var startOfTheYear = new LocalDate(today.Year, 1, 1).AtMidnight();

In [4]:
var createdIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                Filter = IssueFilter.Created
            };
var closedIssuesRequest = new RepositoryIssueRequest
            {
                Since = startOfTheMonth.ToDateTimeUnspecified(),
                State = ItemStateFilter.Closed
            };
var thisYearIssuesRequest = new RepositoryIssueRequest
            {
                State = ItemStateFilter.All,
                Since = startOfTheYear.ToDateTimeUnspecified()
            };

In [5]:
var createdThisMonth = await gitHubClient.Issue.GetAllForRepository("nightscout", "cgm-remote-monitor", createdIssuesRequest);
var closedThisMonth = await gitHubClient.Issue.GetAllForRepository("nightscout", "cgm-remote-monitor", closedIssuesRequest);
var thisYearIssues = await gitHubClient.Issue.GetAllForRepository("nightscout", "cgm-remote-monitor", thisYearIssuesRequest);

In [6]:
var openSoFar = createdThisMonth.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "open").ToArray();
var openByMonthOfCreation = openSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();

var closedSoFar = thisYearIssues.OrderBy(i => i.CreatedAt).Where(i => i.State.StringValue == "closed").ToArray();
var closedByMonthOfClosure = closedSoFar.GroupBy(i => new { i.ClosedAt.Value.Year, i.ClosedAt.Value.Month})
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();
var totalOpenIssues = thisYearIssues.Count();
var openCountByMonth = closedSoFar.GroupBy(i => new { i.CreatedAt.Year, i.CreatedAt.Month})
                .Select(g => {
                    var count = g.Count();                    
                    var dataPoint = new  {Date = g.Key, Count = totalOpenIssues};
                    totalOpenIssues -= count;
                    return dataPoint;
                }).ToArray();

In [7]:
var createdThisMonthByDay = createdThisMonth.GroupBy(i => new DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day)); 
var lineChart = Chart.Line(createdThisMonthByDay.Select(g => new Tuple<DateTime,int>(g.Key, g.Count())));
lineChart.WithTitle("Daily Issues");
lineChart.WithXTitle("Month");
lineChart.WithYTitle("Number of Issues");
display(lineChart);

In [8]:
display(openSoFar.Select(i => new {i.CreatedAt, i.Title, State = i.State.StringValue,  i.Number}).OrderByDescending(d => d.CreatedAt));

index,CreatedAt,Title,State,Number
0,2019-10-24 15:36:44Z,Discord link doesn't do anything,open,5149
1,2019-10-24 05:51:06Z,Fat & Protein Display,open,5146
2,2019-10-24 02:40:01Z,Stop Focus Circles from Eating Entries,open,5145
3,2019-10-23 22:27:37Z,Merge pull request #6 from nightscout/dev,open,5144
4,2019-10-22 01:03:02Z,Update 10.3,open,5139
5,2019-10-21 21:37:29Z,Fix Alexa Rollup Handler,open,5138
6,2019-10-21 21:30:02Z,Update virtual assistants v2,open,5137
7,2019-10-21 20:46:49Z,Jelly Bean 12.3,open,5136
8,2019-10-21 12:44:42Z,Performance optimisation needed,open,5130
9,2019-10-20 22:02:59Z,Adding to #5121,open,5126


In [8]:
var lineChart = Chart.Line(openByMonthOfCreation.Select(g => new Tuple<DateTime, 
                int>(new DateTime(g.Date.Year, g.Date.Month, 1),g.Count)));
lineChart.WithXTitle("Month");
lineChart.WithYTitle("Number of opened issues");
lineChart.WithTitle("Issues still opened grouped by month");
display(lineChart);

In [9]:
var idleByMonth = openSoFar.Where( i => i.Comments == 0).GroupBy(i => new DateTime( i.CreatedAt.Year, i.CreatedAt.Month, 1))
    .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();
var activeByMonth = openSoFar.Where( i => i.Comments > 0).GroupBy(i => new DateTime( i.CreatedAt.Year, i.CreatedAt.Month, 1))
                .Select(g => new {Date = g.Key, Count = g.Count()}).ToArray();

var idleSeries = new Graph.Scattergl
{
    name = "Idle",
    y = idleByMonth.Select(g => g.Count ).ToArray(),
    x = idleByMonth.Select(g => g.Date ).ToArray()
};

var activeSeries = new Graph.Scattergl
{
    name = "Active",
    y = activeByMonth.Select(g => g.Count ).ToArray(),
    x = activeByMonth.Select(g => g.Date ).ToArray()
};


var chart = Chart.Plot(new[] {idleSeries, activeSeries});
chart.WithTitle("Idle and active open issue report");
display(chart);

In [12]:
var openDataPoints = openByMonthOfCreation
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();


var closedDataPoints = closedByMonthOfClosure
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();

var openCountByMonthDataPoints = openCountByMonth
    .Select(g => new { y = g.Count, x = new DateTime(g.Date.Year, g.Date.Month, 1)} )
    .OrderBy(d => d.x).ToArray();

var openSeries = new Graph.Scattergl
{
    name = "Created",
    y = openDataPoints.Select(g => g.y ).ToArray(),
    x = openDataPoints.Select(g => g.x ).ToArray()
};

var closeSeries = new Graph.Scattergl
{
    name = "Closed",
    y = closedDataPoints.Select(g => g.y ).ToArray(),
    x = closedDataPoints.Select(g => g.x ).ToArray()
};

var stillOpenSeries = new Graph.Scattergl
{
    name = "Open",
    y = openCountByMonthDataPoints.Select(g => g.y ).ToArray(),
    x = openCountByMonthDataPoints.Select(g => g.x ).ToArray()
};


var chart = Chart.Plot(new[] {openSeries, closeSeries, stillOpenSeries});
chart.WithTitle("Issue report for the current year");
display(chart);